In [1]:
!pwd

/content


In [16]:
# Importando libs necessárias
import pandas as pd

# Extração

In [2]:
# Montar o diretório compartilhado do Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive

Mounted at /content/gdrive
/content/gdrive


In [5]:
# Listar arquivos do diretório compartilhado
%cd MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
!ls

[Errno 2] No such file or directory: 'MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets'
/content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
bweb_1t_AM_181120201549.zip
bweb_2t_AM_301120201245.zip
detalhe_votacao_secao_2020.zip
eleitorado_local_votacao_2020.zip
perfil_comparecimento_abstencao_2020.zip
perfil_eleitorado_2020.zip
perfil_eleitor_secao_2020_AM.zip
prestacao_contas_anual_partidaria_2020.zip
prestacao_de_contas_eleitorais_candidatos_2020.zip
votacao_candidato_munzona_2020.zip
votacao_secao_2020_AM.zip


# Transformação

In [31]:
!mkdir /content/work #diretório temporário de trabalho
!mkdir /content/datasets #diretório com os arquivos processados

mkdir: cannot create directory ‘/content/work’: File exists


## Boletins de urna

In [8]:
# boletins de urna do 1o turno
!unzip -u bweb_1t_AM_181120201549.zip -d /content/work

Archive:  bweb_1t_AM_181120201549.zip
  inflating: /content/work/bweb_1t_AM_181120201549.csv  
  inflating: /content/work/leiame-boletimurnaweb.pdf  


In [13]:
# boletins de urna do 2o turno
!unzip -u bweb_2t_AM_301120201245.zip -d /content/work

Archive:  bweb_2t_AM_301120201245.zip
  inflating: /content/work/bweb_2t_AM_301120201245.csv  


## Locais de votação

In [15]:
# detalhes das seções
!unzip -u detalhe_votacao_secao_2020.zip -d /content/work

Archive:  detalhe_votacao_secao_2020.zip
  inflating: /content/work/detalhe_votacao_secao_2020_AC.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AM.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AP.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BRASIL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_CE.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_ES.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_GO.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MG.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MS.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MT.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PB.csv  
  inflatin

In [27]:
df_dvs_am = pd.read_csv('/content/work/detalhe_votacao_secao_2020_AM.csv', encoding="ISO-8859-1", delimiter=';')
df_dvs_am.sample(10)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA,QT_VOTOS_ANULADOS_APU_SEP,NR_LOCAL_VOTACAO
1809,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2615,MAUÉS,2615,MAUÉS,5,69,11,PREFEITO,344,253,91,235,3,15,0,0,1180
8714,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,2,427,ELEIÇÕES MUNICIPAIS 2020,29/11/2020,M,AM,2550,MANAUS,2550,MANAUS,31,567,11,PREFEITO,380,337,43,299,12,26,0,0,1988
11497,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2518,LÁBREA,2518,LÁBREA,12,74,13,VEREADOR,370,288,82,269,0,7,12,0,1570
12855,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2534,MANACAPURU,2534,MANACAPURU,6,14,11,PREFEITO,405,314,91,301,4,9,0,0,1716
12071,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2356,HUMAITÁ,2356,HUMAITÁ,17,40,11,PREFEITO,335,283,52,267,5,11,0,0,1457
510,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2259,COARI,2259,COARI,8,121,13,VEREADOR,371,330,41,309,0,3,18,0,1473
997,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,37,531,11,PREFEITO,372,319,53,291,9,19,0,0,1805
1807,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2038,ANORI,2038,ANORI,33,34,11,PREFEITO,401,349,52,340,1,8,0,0,1058
11867,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,62,561,11,PREFEITO,396,305,91,275,12,18,0,0,1740
16198,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,2,427,ELEIÇÕES MUNICIPAIS 2020,29/11/2020,M,AM,2550,MANAUS,2550,MANAUS,2,610,11,PREFEITO,381,313,68,279,6,28,0,0,1880


In [20]:
df_dvs = pd.read_csv('/content/work/detalhe_votacao_secao_2020_BRASIL.csv', encoding="ISO-8859-1", delimiter=';')
df_dvs.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA,QT_VOTOS_ANULADOS_APU_SEP,NR_LOCAL_VOTACAO
336835,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,RJ,58190,CAMPOS DOS GOYTACAZES,58190,CAMPOS DOS GOYTACAZES,75,99,13,VEREADOR,301,224,77,203,3,1,17,0,1562
838591,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,SP,62138,BARUERI,62138,BARUERI,199,47,13,VEREADOR,394,297,97,257,10,9,21,0,1066
575333,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,GO,93009,APARECIDA DO RIO DOCE,93009,APARECIDA DO RIO DOCE,106,95,13,VEREADOR,345,230,115,207,2,7,14,0,1015
482559,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,PE,25135,PAULISTA,25135,PAULISTA,12,200,13,VEREADOR,408,346,62,297,13,22,14,0,1333
282371,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,PI,12246,MILTON BRANDÃO,12246,MILTON BRANDÃO,12,66,13,VEREADOR,232,202,30,186,2,4,10,0,1040


In [29]:
dft_dvs = df_dvs[df_dvs.NM_MUNICIPIO.eq('MANAUS')].sort_values(['DS_CARGO','NR_TURNO','NR_ZONA', 'NR_SECAO']).drop(columns=['DT_GERACAO','HH_GERACAO'])
dft_dvs

,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA,QT_VOTOS_ANULADOS_APU_SEP,NR_LOCAL_VOTACAO
41373,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,1,640,11,PREFEITO,411,348,63,314,15,19,0,0,2429
325496,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,1,641,11,PREFEITO,393,326,67,293,10,23,0,0,2089
772189,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,1,642,11,PREFEITO,397,341,56,309,15,17,0,0,2321
792064,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,1,643,11,PREFEITO,480,414,66,360,17,37,0,0,2054
586504,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,1,644,11,PREFEITO,328,277,51,259,10,8,0,0,2402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346767,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,70,803,13,VEREADOR,275,201,74,167,14,6,14,0,2003
138348,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,70,804,13,VEREADOR,332,228,104,196,18,10,4,0,2038
155963,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,70,805,13,VEREADOR,366,258,108,234,14,8,2,0,2046
812432,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,AM,2550,MANAUS,2550,MANAUS,70,806,13,VEREADOR,399,280,119,248,14,13,5,0,2089


In [33]:
!cp /content/work/leiame.pdf /content/datasets/leiame-detalhe_votacao_secao.pdf
dft_dvs.to_csv('/content/datasets/detalhe_votacao_secao.csv')

In [34]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

# Carga

In [36]:
%cd /content
!tar -czvf eleicoes2020.tar.gz datasets/*

/content
datasets/detalhe_votacao_secao.csv
datasets/leiame-detalhe_votacao_secao.pdf


In [39]:
# sobrescreve o arquivo compartilhado: 1-1v_7xBem5VdYwFFaaNl2EKPLYaDadPa
!cp eleicoes2020.tar.gz /content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final

In [37]:
# remove o diretório
!rm -rf datasets

In [38]:
!tar -zxvf eleicoes2020.tar.gz

datasets/detalhe_votacao_secao.csv
datasets/leiame-detalhe_votacao_secao.pdf
